In [2]:
#import the needed package
import requests
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import NumeralTickFormatter
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure

In [3]:
# Call the PV Watts to get the data (prediction)
nrel_long_tilt_tmy2 = []

# We choose the tilt at the below degree, and the Fairbanks lon & lat are (64.82,-147.87)
tilts = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
for i in range(len(tilts)):
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilts[i], "azimuth": 180, "lat": 60.79, "lon": -161.71, "dataset": 'tmy2'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_long_tilt_tmy2.append(new_dataframe)

# Call the PV Watts to get the data (prediction)
nrel_long_tilt_tmy3 = []

# We choose the tilt at the below degree, and the Fairbanks lon & lat are (64.82,-147.87)
tilts = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
for i in range(len(tilts)):
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilts[i], "azimuth": 180, "lat": 60.79, "lon": -161.71, "dataset": 'tmy3'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_long_tilt_tmy3.append(new_dataframe)

In [4]:
annual_production_tmy2 = []
for i in range(len(tilts)):
    annual_production_tmy2.append(nrel_long_tilt_tmy2[i]['ac_annual'][2])
    
annual_production_tmy3 = []
for i in range(len(tilts)):
    annual_production_tmy3.append(nrel_long_tilt_tmy3[i]['ac_annual'][2])

d_tmy2 = {'Tilts':tilts,'Annual_production':annual_production_tmy2}
df_tmy2 = pd.DataFrame(d_tmy2)

d_tmy3 = {'Tilts':tilts,'Annual_production':annual_production_tmy3}
df_tmy3 = pd.DataFrame(d_tmy3)

#Then find out the max production raw
max_tilt_tmy2 = int(df_tmy2[['Annual_production']].idxmax().values)
max_tilt_tmy3 = int(df_tmy3[['Annual_production']].idxmax().values)


#Then calculate the other tilts' lose compared with the max annual production
lose_tmy2 = []
for index, row in df_tmy2.iterrows():
    tilt_loss = 1- row['Annual_production']/df_tmy2['Annual_production'][max_tilt_tmy2]
    lose_tmy2.append(tilt_loss)

df_tmy2['loss']=lose_tmy2

lose_tmy3 = []
for index, row in df_tmy3.iterrows():
    tilt_loss = 1- row['Annual_production']/df_tmy3['Annual_production'][max_tilt_tmy3]
    lose_tmy3.append(tilt_loss)

df_tmy3['loss']=lose_tmy3

output_file("Bethel_tilts_loss.html")

p = figure(x_axis_label='Tilts', y_axis_label='loss (%)',plot_width=500, plot_height=250)

# add a line renderer
p.line(tilts, df_tmy2['loss'], line_width=2,color='red',legend="TMY2")
p.line(tilts,df_tmy3['loss'],line_width=2,color='blue',legend="TMY3")
p.xaxis.ticker = [10,20,30,40,50,60,70,80,90]
p.yaxis.formatter = NumeralTickFormatter(format='0 %')
p.title.text = "Annual Production loss of different tilts"
p.title.align = "center"
p.title.text_color = "olive"
p.title.text_font = "times"
p.title.text_font_style = "italic"
p.title.text_font_size = '12pt'



show(p)